In [1]:
#hide
#default_exp dev.clientgen

In [1]:
#exports
from jinja2 import Template

from ElexonDataPortal.dev import specgen, rawgen, clientprep

In [2]:
import os
from dotenv import load_dotenv

assert load_dotenv('../.env'), 'Environment variables could not be loaded'

api_key = os.environ['BMRS_API_KEY']

<br>

### API Client Generation

In [3]:
#exports
def generate_streams(API_yaml_fp):
    request_type_to_date_range_example = {
        'SP_and_date': ('2020-01-01', '2020-01-01 1:30'),
        'date_range': ('2020-01-01', '2020-01-07'),
        'date_time_range': ('2020-01-01', '2020-01-07'),
        'year': ('2019-01-01', '2021-01-01'),
        'year_and_month': ('2020-01-01', '2020-06-01'),
        'year_and_week': ('2020-01-01', '2020-06-01'),
        'non_temporal': (None, None)
    }
    
    API_yaml = specgen.load_API_yaml(API_yaml_fp)
    functions = rawgen.construct_all_functions(API_yaml)
    method_info = clientprep.construct_method_info_dict(API_yaml_fp)

    streams = list()

    for function in functions:
        name = function['name']
        function_method_info = method_info[name]

        stream = dict()
        stream['name'] = name
        stream['description'] = function['description']
        stream['date_range_example'] = request_type_to_date_range_example[function_method_info['request_type']]
        stream['extra_kwargs'] = [param for param in function['parameters'] if param['name'] not in list(function_method_info['kwargs_map'].values())+['APIKey', 'ServiceType']]
        stream['request_type'] = function_method_info['request_type']
        stream['kwargs_map'] = function_method_info['kwargs_map']
        stream['func_params'] = list(function_method_info['func_kwargs'].keys())

        streams += [stream]
        
    return streams

In [4]:
API_yaml_fp = '../data/BMRS_API.yaml'

streams = generate_streams(API_yaml_fp)

streams[0]

{'name': 'get_B0610',
 'description': 'Actual Total Load per Bidding Zone',
 'date_range_example': ('2020-01-01', '2020-01-01 1:30'),
 'extra_kwargs': [],
 'request_type': 'SP_and_date',
 'kwargs_map': {'date': 'SettlementDate', 'SP': 'Period'},
 'func_params': ['APIKey', 'date', 'SP', 'ServiceType']}

In [5]:
#exports
def save_api_client(
    API_yaml_fp: str,
    in_fp: str='../templates/api.py',
    out_fp: str='../ElexonDataPortal/api.py'
):
    streams = generate_streams(API_yaml_fp)
    rendered_schema = Template(open(in_fp).read()).render(streams=streams)

    with open(out_fp, 'w') as f:
        try:
            f.write(rendered_schema)
        except e as exc:
            raise exc

In [6]:
save_api_client(API_yaml_fp)

<br>

### API Client Testing

In [8]:
from ElexonDataPortal import api

client = api.Client(api_key)

df = client.get_B1610()

df.head(3)

B1610: 100%|█████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.24s/it]


,documentType,businessType,processType,timeSeriesID,curveType,settlementDate,powerSystemResourceType,registeredResourceEICCode,marketGenerationUnitEICCode,marketGenerationBMUId,...,nGCBMUnitID,activeFlag,documentID,documentRevNum,resolution,start,end,settlementPeriod,quantity,local_datetime
0,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-180,Sequential fixed size block,2020-01-01,Generation,48W000000FASN-42,48W000000FASN-42,E_FASN-4,...,FASN-4,Y,ELX-EMFIP-AGOG-22495386,1,PT30M,2020-01-01,2020-01-01,1,7.46,NaT
1,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-298,Sequential fixed size block,2020-01-01,Generation,48W000000HINB-77,48W000000HINB-77,T_HINB-7,...,HINB-7,Y,ELX-EMFIP-AGOG-22495386,1,PT30M,2020-01-01,2020-01-01,1,502.064,NaT
2,Actual generation,Production,Realised,ELX-EMFIP-AGOG-TS-317,Sequential fixed size block,2020-01-01,Generation,48W00000LARYO-4T,48W00000LARYO-4T,T_LARYW-4,...,LARYO-4,Y,ELX-EMFIP-AGOG-22495386,1,PT30M,2020-01-01,2020-01-01,1,70.118,NaT


In [9]:
client_methods = [method for method in dir(client) if ('__' not in method) and (method not in ['api_key', 'methods'])]
method_to_df = dict()

for client_method in client_methods:
    method_to_df[client_method] = getattr(client, client_method)()

B0650: 100%|█████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.41s/it]
c:\users\ayrto\desktop\phd\data\bmrs\elexon-bmrs-api-wrapper\ElexonDataPortal\dev\utils.py:27: UserWarning: Data request was succesful but no content was returned
  warn(f'Data request was succesful but no content was returned')
DISBSAD: 100%|███████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00,  5.19it/s]
c:\users\ayrto\desktop\phd\data\bmrs\elexon-bmrs-api-wrapper\ElexonDataPortal\dev\orchestrator.py:114: UserWarning: Response was capped: a new `start_date` to continue requesting could not be determined automatically, please handle manually for `get_MID`
  warn(f'Response was capped: a new `start_date` to continue requesting could not be determined automatically, please handle manually for `{method}`')
PHYBMDATA: 100%|█████████████████████████████████████████████████████████████████████████| 4/4 [00:19<00:00,  4.77s/it]
c:\us

In [10]:
method = 'get_B1440'

print(getattr(client, method).__doc__)
method_to_df[method].head(3)


        Generation forecasts for Wind and Solar
        
        Parameters:
            start_date (str)
            end_date (str)
            ProcessType (str)
        


,timeSeriesID,businessType,powerSystemResourceType,settlementDate,processType,settlementPeriod,quantity,documentType,curveType,resolution,activeFlag,documentID,documentRevNum,local_datetime
0,NGET-EMFIP-DGWS-TS-00034592,Solar generation,"""Solar""",2020-01-01,Day Ahead,1,0,Wind and solar forecast,Sequential fixed size block,PT30M,Y,NGET-EMFIP-DGWS-00035923,1,NaT
1,NGET-EMFIP-DGWS-TS-00034590,Wind generation,"""Wind Offshore""",2020-01-01,Day Ahead,1,2843.181,Wind and solar forecast,Sequential fixed size block,PT30M,Y,NGET-EMFIP-DGWS-00035923,1,NaT
2,NGET-EMFIP-DGWS-TS-00034591,Wind generation,"""Wind Onshore""",2020-01-01,Day Ahead,1,3024.238,Wind and solar forecast,Sequential fixed size block,PT30M,Y,NGET-EMFIP-DGWS-00035923,1,NaT


In [1]:
#hide
from ElexonDataPortal.dev.nbdev import notebook2script
notebook2script('05-orchestrator.ipynb')

Converted 05-orchestrator.ipynb.
